
# USA-Sport-clean-data-NFL-MBL-NBA-y-NHL-MLB
## Description
In this script, we will read a file containing metropolitan regions and their associated sports teams from assets/wikipedia_data.html to answer some questions about each metropolitan area. These regions may have one or more teams from the "Big 4" leagues: NFL (football, found in assets/nfl.csv), MLB (baseball, found in assets/mlb.csv), NBA (basketball, found in assets/nba.csv), or NHL (hockey, found in assets/nhl.csv). Remember, all questions should be addressed from the perspective of the metropolitan region, and this file serves as the "authority" for the location of each sports team. Thus, teams commonly known by a different area (e.g., "Oakland Raiders") need to be mapped to the given metropolitan region (e.g., San Francisco Bay Area).

For each sport, please answer the following question: what is the correlation between the win/loss ratio and the population of the city it is in? The win/loss ratio is defined as the number of wins divided by the sum of wins and losses. To calculate the correlation using [pearsonr], you will need to provide two ordered lists of values: the populations from the wikipedia_data.html file and the win/loss ratios for a given sport in the same order. Average the win/loss ratios for cities with multiple teams in a single sport. Each sport contributes equally to this assignment (20%*4=80%) of the total grade. Use only data from the year 2018 for your analysis -- this is crucial!

v

## Stage 1
Win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [ ]:


def nhl_correlation():

    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]] #Selecciona todas las filas exepto la ultima, y se indica los índices de las columnas que se van a conservar



    nhl_df=nhl_df[nhl_df.year==2018] #Dejar solo las columnas del 2018
    nhl_df=nhl_df[["team", "W", "L"]] #Dejar solo las columnas necesarias de NHL
    nhl_df["team"] = nhl_df["team"].str.replace("\*$", "", regex=True) #Limpiar el nombre de los equipos. Algunos equipos tienen "*" al final
    cities["NHL"]=cities["NHL"].replace(r"\[.*\]", "", regex=True) #Limpiar el nombre de los equipos en "cities" de "NHL". Algunos de ellos tienen "[]" en el
    cities=cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']] #Eliminar las columnas inecesarias
    nhl_df["team_only"] = nhl_df.team.apply(lambda x: x.rsplit(None, 1)[-1]) #En nhl_df los nombres de los equipos están mezclados con los nombres de las ciudades. Cepararlos para obtener los ganadores y perdedores
    nhl_df.loc[3, "team_only"] = "Maple Leafs" #Algunos equipos tienen mas de una palabra en el nombre. Corregir eso
    nhl_df.loc[5, "team_only"] = "Red Wings"
    nhl_df.loc[13, "team_only"] = "Blue Jackets"
    nhl_df.loc[27, "team_only"] = "Golden Knights"

    #Remplazar los nombres de los equipos de la misma ciudad

    nhl_df.loc[14, "team_only"] = "Rangers Islanders Devils"
    nhl_df.loc[16, "team_only"] = "Rangers Islanders Devils"
    nhl_df.loc[17, "team_only"] = "Rangers Islanders Devils"

    nhl_df.loc[28, "team_only"] = "Kings Ducks"
    nhl_df.loc[30, "team_only"] = "Kings Ducks"

    cities=cities.rename(columns={'Metropolitan area':'city', 'Population (2016 est.)[8]':'population', 'NHL':'team_only'})#Renombrar las ciudades en un formato mas legible
    nhl_df=nhl_df[~nhl_df.W.str.contains("Division")]#Quitar las filas inecesarias con el nombre "Division" en ellas
    nhl_df.W=nhl_df.W.astype(float) #Cambiar el tipo de datos de ganadores y perdedores de str a float
    nhl_df.L=nhl_df.L.astype(float)
    nhl_df=nhl_df.groupby(by="team_only")["W", "L"].mean() #Hay 4 equipos con el mismo nombre, vamos a agruparlos y obtener su promedio de los valores win y lose
    nhl_df=nhl_df.reset_index() #Restablecer los indices luego de agruparlos para que puedan ser combinados con cities_df con su columna en común
    merged_df=pd.merge(nhl_df, cities, on="team_only") #Combinar ambos dataframes con las columnas requeridas de ambos



    merged_df[["W", "L"]]=merged_df[["W", "L"]].astype(float) #Cambiar el tipo de datos de w y l de str a float



    merged_df["win_lose_ratio"] = merged_df.W/(merged_df.L+merged_df.W) #Crear la coluumna ratio win/lose ratio = win/lose+win

    #Finalmente pase los valores numericos de population_by_region y win_loss_by_region


    population_by_region = pd.to_numeric(merged_df["population"],errors="coerce") # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merged_df["win_lose_ratio"],errors="coerce") # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nhl_correlation()

0.01230899645574427

## Stage 2
Win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:

def nba_correlation():
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]] #Selecciona todas las filas exepto la ultima, y se indica los índices de las columnas que se van a conservar

    nba_df=nba_df[nba_df["year"] == 2018] #Dejar solo las columnas del 2018
    nba_df=nba_df[["team", "W", "L"]]#Dejar solo las columnas necesarias de NHL
    nba_df["team"]=nba_df["team"].str.replace(r"\(.*\)","").str.replace(r"\*","") #Limpiar el nombre de los equipos. Algunos equipos tienen "*" y "()" al final

    cities["NBA"]=cities["NBA"].str.replace("\[.*\]","") #Limpiar el nombre de los equipos en "cities" de "NBA". Algunos de ellos tienen "[]" en el
    cities=cities[["Metropolitan area", "Population (2016 est.)[8]", "NBA"]]
    nba_df["team_only"]=nba_df.team.apply(lambda x: x.rsplit(None, 1)[-1])

    nba_df.loc[10, "team_only"]="Knicks Nets"
    nba_df.loc[11, "team_only"]="Knicks Nets"
    nba_df.loc[24, "team_only"]="Lakers Clippers"
    nba_df.loc[25, "team_only"]="Lakers Clippers"
    nba_df.loc[17, "team_only"]="Trail Blazers"

    cities=cities.rename(columns={'Metropolitan area':'city', 'Population (2016 est.)[8]':'population', 'NBA':'team_only'})#Renombrar las ciudades en un formato mas legible

    nba_df.W=nba_df.W.astype(float) #Cambiar el tipo de datos de ganadores y perdedores de str a float
    nba_df.L=nba_df.L.astype(float)

    nba_df=nba_df.groupby(by="team_only")["W", "L"].mean() #Hay 4 equipos con el mismo nombre, vamos a agruparlos y obtener su promedio de los valores win y lose
    nba_df=nba_df.reset_index() #Restablecer los indices luego de agruparlos para que puedan ser combinados con cities_df con su columna en común
    merged_df=pd.merge(nba_df, cities, on="team_only") #Combinar ambos dataframes con las columnas requeridas de ambos

    merged_df[["W", "L"]]=merged_df[["W", "L"]].astype(float) #Cambiar el tipo de datos de w y l de str a float

    merged_df["win_lose_ratio"] = merged_df.W/(merged_df.L+merged_df.W) #Crear la coluumna ratio win/lose ratio = win/lose+win


    population_by_region = pd.to_numeric(merged_df["population"],errors="coerce") # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merged_df["win_lose_ratio"],errors="coerce") # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

FileNotFoundError: [Errno 2] No such file or directory: 'assets/nba.csv'

## Stage 3
Win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
def mlb_correlation():

    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]] #Selecciona todas las filas exepto la ultima, y se indica los índices de las columnas que se van a conservar

    mlb_df=mlb_df[mlb_df.year==2018] #Dejar solo las columnas del 2018
    mlb_df=mlb_df[["team", "W", "L"]] #Dejar solo las columnas necesarias de NHL
    mlb_df["team"] = mlb_df["team"].str.replace("\*$", "", regex=True) #Limpiar el nombre de los equipos. Algunos equipos tienen "*" al final
    cities["MLB"]=cities["MLB"].replace(r"\[.*\]", "", regex=True) #Limpiar el nombre de los equipos en "cities" de "NHL". Algunos de ellos tienen "[]" en el
    cities=cities[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB']] #Eliminar las columnas inecesarias
    mlb_df["team_only"] = mlb_df.team.apply(lambda x: x.rsplit(None, 1)[-1]) #En nhl_df los nombres de los equipos están mezclados con los nombres de las ciudades. Cepararlos para obtener los ganadores y perdedores


    mlb_df.loc[13, "team_only"] = "Dodgers Angels" #Algunos equipos tienen mas de una palabra en el nombre. Corregir eso
    mlb_df.loc[25, "team_only"] = "Dodgers Angels"

    mlb_df.loc[18, "team_only"] = "Yankees Mets"
    mlb_df.loc[1, "team_only"] = "Yankees Mets"

    mlb_df.loc[11, "team_only"] = "Giants Athletics"
    mlb_df.loc[28, "team_only"] = "Giants Athletics"

    mlb_df.loc[0, "team_only"] = "Red Sox"
    #mlb_df.loc[2, "team_only"] = "Bay Rays"
    mlb_df.loc[3, "team_only"] = "Blue Jays"

    mlb_df.loc[21, "team_only"] = "Cubs White Sox"
    mlb_df.loc[8, "team_only"] = "Cubs White Sox"




    cities=cities.rename(columns={'Metropolitan area':'city', 'Population (2016 est.)[8]':'population', 'MLB':'team_only'})#Renombrar las ciudades en un formato mas legible
    #mlb_df=mlb_df[~mlb_df.W.str.contains("Division")]#Quitar las filas inecesarias con el nombre "Division" en ellas
    mlb_df.W=mlb_df.W.astype(float) #Cambiar el tipo de datos de ganadores y perdedores de str a float
    mlb_df.L=mlb_df.L.astype(float)
    mlb_df=mlb_df.groupby(by="team_only")["W", "L"].mean() #Hay 4 equipos con el mismo nombre, vamos a agruparlos y obtener su promedio de los valores win y lose
    mlb_df=mlb_df.reset_index() #Restablecer los indices luego de agruparlos para que puedan ser combinados con cities_df con su columna en común
    merged_df=pd.merge(mlb_df, cities, on="team_only") #Combinar ambos dataframes con las columnas requeridas de ambos



    merged_df[["W", "L"]]=merged_df[["W", "L"]].astype(float) #Cambiar el tipo de datos de w y l de str a float


    merged_df["win_lose_ratio"] = merged_df.W/(merged_df.L+merged_df.W) #Crear la coluumna ratio win/lose ratio = win/lose+win


       # raise NotImplementedError()

    population_by_region = pd.to_numeric(merged_df["population"],errors="coerce") # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merged_df["win_lose_ratio"],errors="coerce") # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]


    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    #
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

0.15052304487104848

## Stage 4
Win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:


def nfl_correlation():

    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]] #Selecciona todas las filas exepto la ultima, y se indica los índices de las columnas que se van a conservar

    nfl_df=nfl_df[nfl_df.year==2018] #Dejar solo las columnas del 2018
    nfl_df=nfl_df[["team", "W", "L"]] #Dejar solo las columnas necesarias de NHL
    nfl_df["team"] = nfl_df["team"].str.replace("\*$", "", regex=True) #Limpiar el nombre de los equipos. Algunos equipos tienen "*" al final
    nfl_df["team"] = nfl_df["team"].str.replace("\+$", "", regex=True)
    cities["NFL"]=cities["NFL"].replace(r"\[.*\]", "", regex=True) #Limpiar el nombre de los equipos en "cities" de "NHL". Algunos de ellos tienen "[]" en el
    cities=cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']] #Eliminar las columnas inecesarias
    nfl_df["team_only"] = nfl_df.team.apply(lambda x: x.rsplit(None, 1)[-1]) #En nhl_df los nombres de los equipos están mezclados con los nombres de las ciuda
    cities
    nfl_df.loc[17, "team_only"] = "Rams Chargers" #Algunos equipos tienen mas de una palabra en el nombre. Corregir eso
    nfl_df.loc[36, "team_only"] = "Rams Chargers"

    nfl_df.loc[24, "team_only"] = "Giants Jets"
    nfl_df.loc[4, "team_only"] = "Giants Jets"

    nfl_df.loc[19, "team_only"] = "49ers Raiders"
    nfl_df.loc[38, "team_only"] = "49ers Raiders"

    cities=cities.rename(columns={'Metropolitan area':'city', 'Population (2016 est.)[8]':'population', 'NFL':'team_only'})#Renombrar las ciudades en un formato mas legible
    nfl_df=nfl_df[~nfl_df.team.str.contains("AFC")]#Quitar las filas inecesarias con el nombre "Division" en ellas
    nfl_df=nfl_df[~nfl_df.team.str.contains("NFC")]#Quitar las filas inecesarias con el nombre "Division" en ellas
    nfl_df.W=nfl_df.W.astype(float) #Cambiar el tipo de datos de ganadores y perdedores de str a float
    nfl_df.L=nfl_df.L.astype(float)
    nfl_df=nfl_df.groupby(by="team_only")["W", "L"].mean() #Hay 4 equipos con el mismo nombre, vamos a agruparlos y obtener su promedio de los valores win y lose
    nfl_df=nfl_df.reset_index() #Restablecer los indices luego de agruparlos para que puedan ser combinados con cities_df con su columna en común
    merged_df=pd.merge(nfl_df, cities, on="team_only") #Combinar ambos dataframes con las columnas requeridas de ambos


    merged_df[["W", "L"]]=merged_df[["W", "L"]].astype(float) #Cambiar el tipo de datos de w y l de str a float


    merged_df["win_lose_ratio"] = merged_df.W/(merged_df.L+merged_df.W) #Crear la coluumna ratio win/lose ratio = win/lose+win


    population_by_region = pd.to_numeric(merged_df["population"],errors="coerce") # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merged_df["win_lose_ratio"],errors="coerce") # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.00492211214934943